## .json 파일 -> .txt 파일 (YOLO)

In [1]:
import os
import json
import itertools

folder_path = './recycle_sample/label/TL_PET_0'                         # .json 폴더 경로
output_folder = './recycle_sample/new_label'  # .txt 파일로 변환 후 저장할 폴더 경로

# 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        with open(os.path.join(folder_path, filename), 'r') as json_file:
            json_data = json.load(json_file)

            seg_list = []

            for annotation in json_data['annotations']:
                pixel = list(itertools.chain.from_iterable(annotation['segmentation']))
                seg_data = [x / 2048 for x in pixel]
                seg_list.append(seg_data)

            # 파일명에 따라 클래스 번호 지정
            if 'PET' in filename:
                class_number = 1
            elif 'PP' in filename:
                class_number = 2
            elif 'PS' in filename:
                class_number = 3
            else:
                class_number = 0

            output_filename = os.path.splitext(filename)[0] + '.txt'
            output_path = os.path.join(output_folder, output_filename)

            with open(output_path, 'w') as file:
                for data in seg_list:
                    file.write(str(class_number) + ' ')
                    file.write(' '.join(map(str, data)) + '\n')

## Split Data (Train, Validation, Test)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

image_folder = './recycle_sample/image/TS_PET_1'               # 이미지 데이터 폴더 경로
label_folder = './recycle_sample/new_label'    # .txt 변환한 라벨링 데이터 폴더 경로
output_folder = './recycle_sample/training_data'         # 새롭게 저장할 폴더 경로

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 이미지폴더와 라벨링폴더의 파일 리스트 가져오기
image_files = os.listdir(image_folder)
label_files = os.listdir(label_folder)

# 이미지폴더와 라벨링폴더의 파일명에서 확장자 제거 후 정렬
image_names = sorted([os.path.splitext(file)[0] for file in image_files])
label_names = sorted([os.path.splitext(file)[0] for file in label_files])

# train, val, test로 데이터 분리
image_train, image_val_test, label_train, label_val_test = train_test_split(image_names, label_names, test_size=0.25, random_state=42)
image_val, image_test, label_val, label_test = train_test_split(image_val_test, label_val_test, test_size=0.25, random_state=42)

# 이전에 생성한 output_folder 안에 images, labels 폴더를 생성하고 그 안에 train, val, test 폴더 생성
for folder in ['images', 'labels']:
    folder_path = os.path.join(output_folder, folder)
    os.makedirs(os.path.join(folder_path, 'train'), exist_ok=True)
    os.makedirs(os.path.join(folder_path, 'val'), exist_ok=True)
    os.makedirs(os.path.join(folder_path, 'test'), exist_ok=True)

# 파일 복사
for image_name, label_name in zip(image_train, label_train):
    shutil.copy(os.path.join(image_folder, image_name + '.jpg'), os.path.join(output_folder, 'images', 'train', image_name + '.jpg'))
    shutil.copy(os.path.join(label_folder, label_name + '.txt'), os.path.join(output_folder, 'labels', 'train', label_name + '.txt'))

for image_name, label_name in zip(image_val, label_val):
    shutil.copy(os.path.join(image_folder, image_name + '.jpg'), os.path.join(output_folder, 'images', 'val', image_name + '.jpg'))
    shutil.copy(os.path.join(label_folder, label_name + '.txt'), os.path.join(output_folder, 'labels', 'val', label_name + '.txt'))

for image_name, label_name in zip(image_test, label_test):
    shutil.copy(os.path.join(image_folder, image_name + '.jpg'), os.path.join(output_folder, 'images', 'test', image_name + '.jpg'))
    shutil.copy(os.path.join(label_folder, label_name + '.txt'), os.path.join(output_folder, 'labels', 'test', label_name + '.txt'))